## Introduction to Timeslot 2

This notebook presents the **transformation and loading steps** (ETL) for job data as part of **Timeslot 2 of the Data Engineering track**.

In **Timeslot 1**, I implemented a robust job scraping tool that:
- Collected job listings from a job website,
- Extracted structured data fields including:
  - `id`
  - `url`
  - `title`
  - `publication_date`
  - `workload`
  - `contract_type`
  - `salary`
  - `languages`
  - `place_of_work`
- Stored the data in multiple formats: **CSV**, **JSON**, and **MySQL**,
- Downloaded and stored the corresponding **HTML files** for each job description.

Below are code snippets from the Timeslot 1 implementation:
```python
def scrape_job(self, job_url):
        """
        Scrape details from a job listing page.
        :param job_url: URL of the job page.
        """
        with self.lock:
            if job_url in self.seen_jobs:
                logging.warning(f"Skipping duplicate job: {job_url}")
                return None  # Skip duplicate scraping
            self.seen_jobs.add(job_url)  # Mark as scraped **inside lock**

        job_html = Fetcher().fetch_page(job_url)
        if not job_html:
            logging.error(f"Failed to fetch job page: {job_url}")
            return None  # Skip failed jobs

        soup = BeautifulSoup(job_html, "html.parser")

        with self.lock:
            job_details = {
                "id": self.job_id,
                "url": job_url,
                "title": self.get_text(soup, "title"),
                "publication_date": self.get_text(soup, "Publication date:") or "N/A",
                "workload": self.get_text(soup, "Workload:") or "N/A",
                "contract_type": self.get_text(soup, "Contract type:") or "N/A",
                "salary": self.get_text(soup, "Salary:") or "N/A",
                "languages": self.get_text(soup, "Language:") or "N/A",
                "place_of_work": self.get_text(soup, "Place of work:") or "N/A"
            }
            self.job_id += 1  # Increment ID

        logging.info(f"Scraped job data: {job_details}")

        self.progress.update_scrape()  # Update progress
        self.scraped_jobs.append(job_details)


        return job_details
```
Some helper methods have been used. the complete class can be seen in /src/scraper.py.

Saving the scraped data to csv is shown below:
```python
def save_to_csv(self, job_list):
        """Saves job data to a CSV file."""
        if not job_list:
            logging.warning("No job data to save.")
            return

        fields = [
            "id", "url", "title", "publication_date", "workload",
            "contract_type", "salary", "languages", "place_of_work"
        ]

        with self.lock:  # Prevent multiple threads from writing simultaneously
            try:
                # Check if the file is empty
                file_empty = not os.path.exists(self.csv_path) or os.path.getsize(self.csv_path) == 0

                with open(self.csv_path, mode="a", newline="", encoding="utf-8") as f:
                    writer = csv.DictWriter(f, fieldnames=fields)

                    # Write the header **ONLY IF** the file is empty
                    if file_empty:
                        writer.writeheader()

                    # Write job data
                    for job in job_list:
                        writer.writerow({field: job.get(field, "N/A") for field in fields})

                logging.info(f"Saved {len(job_list)} jobs to CSV: {self.csv_path}")


            except Exception as e:
                logging.error(f"Error saving to CSV: {e}")
```
Some helper methods have been used and other direct saving methods to "JSON" and "mySQL" were implemented in Timeslot 1 and can be found in /src/data_saver.py.

This code handled the full pipeline for downloading, extracting, and storing job data. Since this step was already implemented in Slot 1, this notebook focuses on adapting the output of that pipeline for transformation and loading into a DuckDB database.

## Objective of This Notebook

For **Timeslot 2**, the goal is to transform the already collected job data and load it into a **DuckDB** database in a clean and structured format, ready for efficient querying and analysis.

To achieve this, I follow two strategies:

1. **Adapt the scraper (**Timeslot 1**) for DuckDB**
   The existing scraping tool has been updated to optionally write output directly to DuckDB instead of MySQL.
   In /src/data_saver.py the function "save_to_sqlite()" is updated to store the scraped data directly to DuckDB. The new function `save_to_duckdb()` replaces `save_to_sqlite()` and does not manually insert the `id`, as DuckDB can handle identifiers internally if needed.

    Snipped of the duckDB output after scraping:


In [18]:
import duckdb
from IPython.display import display

# Connect to your DuckDB database file
con = duckdb.connect("../data/jobs.duckdb")  # Adjust path

# Load rows from the job_descriptions table
# Adjust "LIMIT X" to see more or less entries
query = """
SELECT
    title,
    url,
    publication_date,
    workload,
    contract_type,
    salary,
    languages,
    place_of_work
FROM job_descriptions
LIMIT 5
"""
df = con.execute(query).fetchdf()

# Optional: replace N/A for aesthetics
df.replace("N/A", "", inplace=True)

# Close connection if not needed later
con.close()

# Format and display in notebook
display(df.style.set_properties(**{
    'text-align': 'left',
    'font-size': '10px'
}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left'), ('font-weight', 'bold')]
}]))

,title,url,publication_date,workload,contract_type,salary,languages,place_of_work
0,Elektroniker als Kalibriertechniker für elektrische Kalibrierung (m/w),https://www.jobs.ch/en/vacancies/detail/3b3fd6d4-5682-4cc4-9d65-938da4dfc738/,25 March 2025,100%,Unlimited employment,CHF 70 000 - 95 000 /year,"English (Intermediate), German (Fluent)",
1,Mitarbeiter Qualitätssicherung Elektronik (m/w/d),https://www.jobs.ch/en/vacancies/detail/fa87ef4d-d8ab-4d8c-9e6a-528629fb1e31/,14 March 2025,100%,Unlimited employment,,German (Fluent),Zürich
2,Elektroniker. 100%.,https://www.jobs.ch/en/vacancies/detail/890f72cb-6951-4973-bdf7-81b36f388f17/,12 March 2025,100%,Unlimited employment,,"German (Fluent), English (Fluent)",
3,Elektroniker / Elektronik-Techniker 100% (m/w),https://www.jobs.ch/en/vacancies/detail/ae3226da-b6c5-48b8-9cfc-d8299dc5dd2c/,05 March 2025,100%,Unlimited employment,,"German (Native language), English (Basic knowledge)",
4,Elektroniker*in 100%,https://www.jobs.ch/en/vacancies/detail/7528c1d9-8c40-4b59-8ae3-a7647677a67b/,03 March 2025,100%,Unlimited employment,,"German (Fluent), English (Fluent)",



2. **Transform and load existing data**
   As the full dataset (over 9'000 job entries) was already saved as a CSV file in Slot 1, this notebook also demonstrates how to import that CSV into DuckDB and display results directly.

---

In the following sections, I will:

- Define the target table structure in DuckDB,
- Import the existing CSV dataset into DuckDB,
- Preview the loaded data and perform simple test queries to validate the import,
- Reflect on this approach and its flexibility.

## Create DuckDB from already scraped data in CSV format:
Adapt the following code, to choose the correct CSV file. A new DuckDB will be created with it.

In [19]:
import duckdb

# Connect to (or create) a new DuckDB database
con = duckdb.connect("../data/jobs_from_csv.duckdb")  # creates a new file

# Drop the table if it already exists
con.execute("DROP TABLE IF EXISTS job_descriptions")

# Create the target table
con.execute("""
    CREATE TABLE job_descriptions (
        id INTEGER,
        url TEXT,
        title TEXT,
        publication_date TEXT,
        workload TEXT,
        contract_type TEXT,
        salary TEXT,
        languages TEXT,
        place_of_work TEXT
    )
""")

# Import CSV into the new DuckDB table
con.execute("""
    INSERT INTO job_descriptions
    SELECT * FROM read_csv_auto('../data/job_descriptions-9492.csv', HEADER=TRUE)
""")

df = con.execute("SELECT * FROM job_descriptions").fetchdf()
df

# In the following Table (top left corner) you can choose how many rows you would like to see. Max with this dataset is 9000+ rows!

,id,url,title,publication_date,workload,contract_type,salary,languages,place_of_work
0,1,https://www.jobs.ch/en/vacancies/detail/57f88c...,Elektroniker EFZ / Auftragsbearbeitung von A-Z...,03 March 2025,80 – 100%,Unlimited employment,N/A,"German (Native language), English (Intermediate)",Regensdorf
1,2,https://www.jobs.ch/en/vacancies/detail/ae3226...,Elektroniker / Elektronik-Techniker 100% (m/w),05 March 2025,100%,Unlimited employment,N/A,"German (Native language), English (Basic knowl...",N/A
2,3,https://www.jobs.ch/en/vacancies/detail/890f72...,Elektroniker. 100%.,12 March 2025,100%,Unlimited employment,N/A,"German (Fluent), English (Fluent)",N/A
3,4,https://www.jobs.ch/en/vacancies/detail/7528c1...,Elektroniker*in 100%,03 March 2025,100%,Unlimited employment,N/A,"German (Fluent), English (Fluent)",N/A
4,5,https://www.jobs.ch/en/vacancies/detail/35ce2c...,Automationsingenieur FH - Life Sciences,28 February 2025,80 – 100%,Unlimited employment,N/A,"German (Fluent), English (Intermediate)",Grossraum Basel
...,...,...,...,...,...,...,...,...,...
9439,9488,https://www.jobs.ch/en/vacancies/detail/e949ec...,Assistant-e projets neufs - Mission temporaire...,16 March 2025,100%,Temporary,N/A,French (Fluent),N/A
9440,9489,https://www.jobs.ch/en/vacancies/detail/953d03...,Responsable Eligreen CDI (H/F),16 March 2025,100%,Unlimited employment,N/A,N/A,N/A
9441,9490,https://www.jobs.ch/en/vacancies/detail/3d87da...,Commercial Project Manager,12 March 2025,5%,Unlimited employment,N/A,English (Fluent),N/A
9442,9491,https://www.jobs.ch/en/vacancies/detail/9a6f12...,Courtier-ère immobilier confirmé-e résidentiel...,16 March 2025,100%,Unlimited employment,N/A,N/A,N/A


Instead of using screenshots, this notebook demonstrates how the data is extracted and displayed directly from the DuckDB database using SQL queries.

The database was created from previously scraped job data and is structured in a simple and effective way: it consists of a single table named job_descriptions, with column headers representing the extracted fields such as title, url, workload, and others.

Since each row corresponds to a single job entry and all relevant data points are contained in that row, splitting the data across multiple tables is unnecessary. This flat structure is ideal for analysis, querying, and transformation tasks in the context of this project.

The created DuckDB database includes a single table `job_descriptions`, as the data schema is already flat and well-structured. This reduces redundancy and simplifies querying. Since each row contains a complete job entry, normalization into multiple tables is not required at this stage.

This structure aligns with the ETL goal of Timeslot 2. Since all extracted job data can be stored in a single row per entry, no normalization or multiple-table design is needed. The flat schema is ideal for analysis in upcoming tasks.

## Conclusion

In this notebook, the transformation and loading steps for Timeslot 2 were successfully implemented.

The CSV data collected during Timeslot 1 was cleaned and loaded into a DuckDB database. The process included table creation, importing and validating the results using simple SQL queries.

Using DuckDB as the storage backend enables high-performance queries without the need for setting up a server or database engine, making it ideal for quick local data exploration and analysis.

This setup will allow further exploration of job market trends in Switzerland in upcoming tasks (e.g., Timeslot 3 or data analytics steps).

## References

- Clément Mihailescu (2024). BeautifulSoup scraping examples and patterns. https://realpython.com/beautiful-soup-web-scraper-python/
- DuckDB DuckDB Documentation and SQL Syntax. https://duckdb.org/docs/
- BFH Data Engineering Course Material (2024HS & 2025FS). Task descriptions for Timeslot 1 & timeslot 2.
- OpenAI ChatGPT. Structure review of markdown and formatting guidance.